In [2]:
import cupy as cp

In [3]:
def normalize_sequential(image_array):
    """
    Normaliza os pixels de uma imagem.

    Parâmetros:
    image_array (numpy array): Array numpy da imagem a ser normalizada.

    Retorna:
    numpy array: O array da imagem normalizada.
    """
    
    image_array = image_array.astype(np.float32)
    
    xmin, xmax = image_array.min(), image_array.max()

    if xmin == xmax:
        raise ValueError("xmin e xmax não podem ser iguais, pois causariam uma divisão por zero.")
    
    normalized_array = (image_array - xmin) / (xmax - xmin)
    
    # Converte o array normalizado de volta para a faixa de 0-255
    normalized_array = (normalized_array * 255).astype(np.uint8)
    
    return normalized_array

In [4]:
# Define um kernel CUDA customizado usando RawKernel
kernel_code = '''
extern "C" __global__
void normalize_kernel(float* image, float xmin, float xmax, float* result, int size) {
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    if (idx < size) {
        result[idx] = (image[idx] - xmin) / (xmax - xmin) * 255.0;
    }
}
'''

normalize_kernel = cp.RawKernel(kernel_code, 'normalize_kernel')

def normalize_custom(image_array):
    """
    Normaliza os pixels de uma imagem usando um kernel CUDA customizado.

    Parâmetros:
    image_array (cupy array): Array da imagem a ser normalizada.

    Retorna:
    cupy array: O array da imagem normalizada.
    """
    image_array = cp.asarray(image_array, dtype=cp.float32)
    xmin, xmax = cp.min(image_array), cp.max(image_array)

    if xmin == xmax:
        raise ValueError("xmin e xmax não podem ser iguais, pois causariam uma divisão por zero.")

    normalized_array = cp.empty_like(image_array)

    size = image_array.size
    block_size = 256
    grid_size = (size + block_size - 1) // block_size

    normalize_kernel((grid_size,), (block_size,), (image_array, xmin, xmax, normalized_array, size))

    return normalized_array.astype(cp.uint8)

In [8]:
def normalize_s(image_array):
    """
    Normaliza os pixels de uma imagem.

    Parâmetros:
    image_array (numpy array): Array numpy da imagem a ser normalizada.

    Retorna:
    numpy array: O array da imagem normalizada.
    """
    
    image_array = image_array.astype(np.float32)
    
    # Inicializa xmin e xmax com valores extremos
    xmin = float('inf')
    xmax = float('-inf')

    # Itera através de todos os elementos do array para encontrar xmin e xmax
    for row in image_array:
        for pixel in row:
            if pixel < xmin:
                xmin = pixel
            if pixel > xmax:
                xmax = pixel

    if xmin == xmax:
        raise ValueError("xmin e xmax não podem ser iguais, pois causariam uma divisão por zero.")
    
    normalized_array = (image_array - xmin) / (xmax - xmin)
    
    # Converte o array normalizado de volta para a faixa de 0-255
    normalized_array = (normalized_array * 255).astype(np.uint8)
    
    return normalized_array

In [12]:
import numpy as np

rows, cols = 10000, 10000

# Criando o array com valores aleatórios entre 0 e 255
large_array = np.random.randint(0, 256, (rows, cols), dtype=np.uint8)

In [14]:
%%timeit -n 100 -r 5
normalize_sequential(large_array)

430 ms ± 10.3 ms per loop (mean ± std. dev. of 5 runs, 100 loops each)


In [15]:
%%timeit
normalize_custom(large_array)

180 ms ± 8.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
